In [1]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#load the claims data

data=pd.read_csv(r"final_data.csv",header=0)
data_bk=data.copy()
data.head()

,Location,MinTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Temp3pm,RainToday,RainTomorrow,Season
0,0.041667,0.535620,0.021898,0.866667,0.527778,0.866667,0.933333,0.512821,0.545455,0.666667,0.212121,0.253102,0.502439,0,0,0.4
1,0.041667,0.377309,0.000000,0.933333,0.527778,0.400000,1.000000,0.102564,0.500000,0.356322,0.242424,0.325062,0.563415,0,0,0.4
2,0.041667,0.522427,0.000000,1.000000,0.555556,0.866667,1.000000,0.487179,0.590909,0.287356,0.292929,0.250620,0.536585,0,0,0.4
3,0.041667,0.424802,0.000000,0.266667,0.250000,0.600000,0.000000,0.282051,0.204545,0.367816,0.151515,0.498759,0.617073,0,0,0.4
4,0.041667,0.643799,0.036496,0.866667,0.486111,0.066667,0.466667,0.179487,0.454545,0.793103,0.323232,0.330025,0.695122,0,0,0.4


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138472 entries, 0 to 138471
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       138472 non-null  float64
 1   MinTemp        138472 non-null  float64
 2   Rainfall       138472 non-null  float64
 3   WindGustDir    138472 non-null  float64
 4   WindGustSpeed  138472 non-null  float64
 5   WindDir9am     138472 non-null  float64
 6   WindDir3pm     138472 non-null  float64
 7   WindSpeed9am   138472 non-null  float64
 8   WindSpeed3pm   138472 non-null  float64
 9   Humidity9am    138472 non-null  float64
 10  Humidity3pm    138472 non-null  float64
 11  Pressure9am    138472 non-null  float64
 12  Temp3pm        138472 non-null  float64
 13  RainToday      138472 non-null  int64  
 14  RainTomorrow   138472 non-null  int64  
 15  Season         138472 non-null  float64
dtypes: float64(14), int64(2)
memory usage: 16.9 MB


In [4]:
indepvar=[]
for col in data.columns:
    if col!='RainTomorrow':
        indepvar.append(col)
depvar='RainTomorrow'
x=data[indepvar]
y=data[depvar]

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((96930, 15), (41542, 15), (96930,), (41542,))

Boosting algorithms on RandomForest

In [9]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

# Define boosting models
boosting_models = {
    'CatBoost': CatBoostClassifier(verbose=0),
    'XGBoost': XGBClassifier(verbosity=0),
    'AdaBoost': AdaBoostClassifier(),
    'LightGBM': LGBMClassifier(verbose=-1),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

def calculate_metrics(y_true, y_pred, y_pred_prob):
    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'ROC AUC': roc_auc_score(y_true, y_pred_prob)
    }
    return metrics

# Create an empty list to store the results
emresults = []

# Loop through each boosting model
for boosting_model_name, boosting_model in boosting_models.items():
    # Train the model on the training dataset
    boosting_model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = boosting_model.predict(x_test)
    y_pred_prob = boosting_model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    metrics_dict = calculate_metrics(y_test, y_pred, y_pred_prob)

    # Create a result dictionary and append to the list
    result = {
        'Model': boosting_model_name,
        **metrics_dict
    }
    emresults.append(result)

# Convert results to DataFrame and display
results_df = pd.DataFrame(emresults)



C:\Users\Maha\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [10]:
results_df

,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,CatBoost,0.859901,0.730544,0.489372,0.586119,0.876063
1,XGBoost,0.856579,0.711634,0.491747,0.581601,0.870335
2,AdaBoost,0.846782,0.700781,0.426078,0.529946,0.841206
3,LightGBM,0.855688,0.719746,0.471797,0.569973,0.868475
4,GradientBoostingClassifier,0.850681,0.720565,0.430234,0.538776,0.853804
